# Explore GRS images

## Load python libraries

In [ ]:
import glob
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import rioxarray  # activate the rio accessor

#import cartopy
import cartopy.crs as ccrs
#import cartopy.feature as cfeature
#import locale
#locale.setlocale(locale.LC_ALL, 'en_US.utf8')
import matplotlib.pyplot as plt
import matplotlib as mpl
#mpl.use('TkAgg')


import pyproj as ppj
import rasterio
from affine import Affine
from shapely.geometry import box,Point, mapping
from shapely.ops import transform


import grstbx
from grstbx import visual

u = grstbx.utils
opj = os.path.join

grstbx.__version__

In [ ]:
import hvplot.xarray

import holoviews as hv
import holoviews.operation.datashader as hd
from holoviews import opts
hv.extension('bokeh')
import datashader as ds
from datashader import transfer_functions as tf 
import colorcet as cc
import panel as pn
import panel.widgets as pnw
#import ipywidgets as widgets


## Set Dask local cluster

In [ ]:
import dask.distributed
cluster = dask.distributed.LocalCluster()
client = dask.distributed.Client(cluster)
cluster.dashboard_link

## Set PROJ path if necessary

In [ ]:
ppj.datadir.get_data_dir()
#ppj.datadir.set_data_dir('/work/scratch/harmelt/envs/grstbx/share/proj')

## Set the images you want to play with

In [ ]:

tile='37PGN'
idir='/media/harmel/vol1/Dropbox/satellite/S2'
select =grstbx.select_files()

# if you need to change the root path
select.root=idir
select.list_tile(tile=tile,product='cnes',pattern='*15.nc')
files = select.list_file_path(('2022-11','2022-12-31'))
files

pn.widgets.DataFrame(select.file_list, height=250,width=1000)

## Load and subset image series

In [ ]:
# central coordinate
lon, lat = 41.77, 11.18 
# size in meter of the rectangle of interest
width, height = 13000, 15000
aspect=width/height

ust = grstbx.utils.spatiotemp()
box = ust.wktbox(lon,lat, width=width, height=height, ellps='WGS84')
bbox = gpd.GeoSeries.from_wkt([box]).set_crs(epsg=4326)
# reproject lon lat in xy coordinates
#bbox = bbox.to_crs(epsg=32631)

# generate datacube
dc = grstbx.l2grs(files)
dc.load(subset=bbox,reshape=False, reproject=False)

## Check data/metadata

In [ ]:
masking_ = grstbx.masking(dc.datacube)


In [ ]:
dc.datacube

## Mask datacube
Mask pixels from chosen flags and remove empty dates

In [ ]:
mask = (masking_.get_mask(high_nir=True) | masking_.get_mask(hicld=True)) | (dc.datacube.Rrs_B3 < 0.0002)
#mask.compute()


In [ ]:
# compute number of valid pixels per date
mask_ts =  (~mask).sum(['x','y']).compute()



In [ ]:
valid_pix = mask_ts / mask_ts.max()
valid_pix.plot()

Drop dates with valid pixel below a threshold, for example keep date when valid pixels are at least 60%

In [ ]:
threshold = 0.6
dc.datacube = dc.datacube.where(valid_pix>threshold,drop=True)



Mask raster for each remaining date

In [ ]:
dc.datacube = dc.datacube.where(mask==0,drop=True)

reproject desired variables into desired epsg (by default 3857 for basemap projection)

In [ ]:
dc.reproject_data_vars()

reshape your raster to add the wavelength diemension to the requested Rrs

In [ ]:
dc.reshape_raster()

## **Fast checking of the RGB images**

You are done with your datacube:

In [ ]:
coarsening = 3
fig = dc.raster.Rrs.isel(wl=2)[:,::coarsening, ::coarsening].plot(col='time', col_wrap=4,robust=True,cmap=cc.cm.CET_D13,aspect=aspect)
for ax in fig.axs.flat:
    ax.set(xticks=[], yticks=[])
    ax.set_ylabel('')
    ax.set_xlabel('')
fig


In [ ]:
coarsening = 2
fig=dc.raster.BRDFg[:,::coarsening, ::coarsening].plot(col='time', col_wrap=4,vmax=0.01,robust=True,cmap=cc.cm.gray,aspect=aspect)
for ax in fig.axs.flat:
    ax.set(xticks=[], yticks=[])
    ax.set_ylabel('')
    ax.set_xlabel('')
fig


In [ ]:
bands=[4,2,1]
#bands=[3,2,1]
coarsening = 2
brightness_factor = 5
gamma=2
fig = (dc.raster.Rrs.isel(wl=bands)[:,:,::coarsening, ::coarsening]**(1/gamma)*brightness_factor).plot.imshow(col='time', col_wrap=4,robust=True,aspect=aspect)
for ax in fig.axs.flat:
    ax.set(xticks=[], yticks=[])
    ax.set_ylabel('')
    ax.set_xlabel('')
fig


## Export your data cube into netcdf (you can try other format, not tested so far)

In [ ]:
ds = dc.raster
comp = dict(zlib=True, complevel=5)
encoding = {'Rrs':comp} #{var: comp for var in ds.data_vars}
filename='/sat_data/satellite/sentinel2/L3/test_abhe_2022_nov-dec.nc'
ds.to_netcdf(filename, encoding=encoding)

## **Check spectral datacube** (i.e., Remote Sensing Reflectance, R<sub>rs</sub>, sr<sup>-1</sup>)

To quickly check your data visually, you can use the *visual* module of *grstbx*

In [ ]:
v=visual.view_spectral(dc.raster.Rrs)

In [ ]:
v.visu()


In [ ]:
v.aoi_stream.data

In [ ]:
v.aoi_stream.element.geom()

In [ ]:
from shapely.geometry import Polygon
geom = v.aoi_stream.data
ys,xs=geom['ys'][-1],geom['xs'][-1]
polygon_geom = Polygon(zip(xs, ys))
polygon = gpd.GeoDataFrame(index=[0], crs=3857, geometry=[polygon_geom]) 
geom_ = polygon.to_crs(3857)#Rrs_masked.rio.crs) #4326)#32631)#4326)
geom_.to_crs(4326).bounds

# if you want to subset within the box (defined from the polygon) uncomment the following lines
#minx, miny, maxx, maxy = geom_.bounds.values[0]
#Rrs_masked.sel(x=slice(minx, maxx), y=slice(maxy, miny))

# if projected uncomment
#geom_=geom_.to_crs(Rrs_masked.rio.crs)


In [ ]:
Rrs_clipped = Rrs_masked.rio.clip(geom_.geometry.values)

In [ ]:
Rrs_clipped = Rrs_masked

In [ ]:
comp = dict(zlib=True, complevel=5)
encoding = {var: comp for var in ds.data_vars}
filename='/sat_data/satellite/sentinel2/L3/test_abhe_2022.nc'
Rrs_clipped.to_netcdf(filename, encoding=encoding)

In [ ]:
stacked = Rrs_clipped.dropna('time',thresh=0).stack(gridcell=["y", "x"]).dropna('gridcell',thresh=0)


In [ ]:
group_coord ='wl'
stat_coord='gridcell'
stats = xr.Dataset({'median':stacked.groupby(group_coord).median(stat_coord)})
stats['q25'] = stacked.groupby(group_coord).quantile(0.25,dim=stat_coord)
stats['q75'] = stacked.groupby(group_coord).quantile(0.75,dim=stat_coord)
stats['min'] = stacked.groupby(group_coord).min(stat_coord)
stats['max'] = stacked.groupby(group_coord).max(stat_coord)
stats['mean'] = stacked.groupby(group_coord).mean(stat_coord)
stats['std'] = stacked.groupby(group_coord).std(stat_coord)
stats['pix_num'] = stacked.count(stat_coord)

In [ ]:
%matplotlib inline
num_items = len(stats.time)
col_wrap=4
rows=int(np.ceil(num_items/col_wrap))

fig, axs = plt.subplots(nrows=rows,ncols=col_wrap, sharex=True,sharey=True, figsize=(20, rows*3.5))#,sharey=True
fig.subplots_adjust(hspace=0.1,wspace=0.1)
axs_ = axs.ravel()
[axi.set_axis_off() for axi in axs_]
for iax,(_,group) in enumerate(stats.groupby('time')): 
    date = group.time.dt.date.values
    
    axs_[iax].set_axis_on()
    axs_[iax].axhline(y=0,color='k',lw=1)
    axs_[iax].plot(group.wl,group['median'],c='k')
    axs_[iax].plot(group.wl,group['mean'],c='red',ls='--')
    axs_[iax].fill_between(group.wl, group['q25'], group['q75'],alpha=0.3,color='grey')
    axs_[iax].set_title(date)
plt.show()

In [ ]:
from datetime import datetime
raster = Rrs_masked#.time.values
dates=raster.time.dt.date.values#.to_pandas() #.astype(datetime)#dt.date.values


dates#[0].datetime#.to_datetime()

In [ ]:
datetime_picker = pn.widgets.DatetimePicker()#,start=dates[0])#,
                                  #  end=dates[-1], 
                                   # value=dates[0])#, enabled_dates=(datetime.datetime(2021, 3, 2, 12, 10)))#dates)
                                          #  value=)

In [ ]:

dataarrays, datasets, images, minmaxes = {}, {}, {}, {}
bands = raster.wl.values

for itime, time in enumerate(raster.time.values):
    dataarrays_={}
    raster_ =raster.sel(time=time)
    for iband,band in enumerate(bands):
        dataarrays[itime,iband] = raster_.sel(wl=band).rio.reproject(3857,nodata=np.nan )
    #dataarrays[itime]=dataarrays_
    

In [ ]:
image_width, image_height = 400, 600
map_width, map_height = 1200, 700

key_dimensions = ['x', 'y']


import bokeh 
formatter_code = """
  var digits = 4;
  var projections = Bokeh.require("core/util/projections");
  var x = special_vars.x; var y = special_vars.y;
  var coords = projections.wgs84_mercator.invert(x, y);
  return "" + (Math.round(coords[%d] * 10**digits) / 10**digits).toFixed(digits)+ "";
"""
formatter_code_x, formatter_code_y = formatter_code % 0, formatter_code % 1
custom_tooltips = [ ('Lon', '@x{custom}'), ('Lat', '@y{custom}'), ('Value', '@image{0.0000}')]
custom_formatters = { 
  '@x' : bokeh.models.CustomJSHover(code=formatter_code_x), 
  '@y' : bokeh.models.CustomJSHover(code=formatter_code_y)
}
custom_hover = bokeh.models.HoverTool(tooltips=custom_tooltips, formatters=custom_formatters)


hv.opts.defaults(
  hv.opts.Image(height=image_height, width=image_width, 
                colorbar=True, tools=[custom_hover], active_tools=['wheel_zoom'], 
                clipping_colors={'NaN': '#00000000'}),
  hv.opts.Tiles(active_tools=['wheel_zoom'])
)
gopts = hv.opts.Tiles(xaxis=None, yaxis=None, bgcolor='black', show_grid=False)

titles ={}
for idate,date in enumerate(dates):
    for iband,band in enumerate(bands):

        titles[date,iband]=str(date)+', wl = {:.2f} nm '.format(band)
        datasets = hv.Dataset(dataarrays[idate,iband].squeeze(),  kdims=key_dimensions)
        images[date,iband] = hv.Image(datasets).opts(gopts)

In [ ]:
minmax=(0,0.02)
bases = [name for name, ts in hv.element.tiles.tile_sources.items()]
pn_band = pn.widgets.RadioButtonGroup(value=2,options=list(range(len(bands))))
pn_colormap = pn.widgets.Select( value='CET_D13',options=['kbc','blues_r', 'kb', 'rainbow', 'fire', 'kgy','bky','bjy','CET_D13'])
pn_opacity = pn.widgets.FloatSlider(name='Opacity', value=0.95, start=0, end=1, step=0.05) 
range_slider = pn.widgets.RangeSlider(name='Range Slider', start=0, end=.06,value=minmax,step=0.0001)
pn_basemaps = pn.widgets.Select(value='StamenTerrainRetina',options=bases)
pn_date = pn.widgets.DatePicker(value=dates[0],start=dates[0],enabled_dates=dates.tolist())#.date, end=dates[-1],value=dates[0])

@pn.depends(
  pn_date_value=pn_date.param.value,
  pn_band_value = pn_band.param.value, 
  pn_colormap_value = pn_colormap.param.value, 
  pn_opacity_value = pn_opacity.param.value,
  range_slider_value = range_slider.param.value
    
)
def load_map(pn_date_value,pn_band_value, pn_colormap_value, pn_opacity_value,range_slider_value):
    
    image = images[pn_date_value,pn_band_value]
    
    used_colormap = cc.cm[pn_colormap_value]
    
    image.opts(cmap=used_colormap, alpha=pn_opacity_value,clim=range_slider_value,title= titles[pn_date_value,pn_band_value])
    return image

@pn.depends(
    basemap_value=pn_basemaps.param.value)
def load_tiles(basemap_value):
    tiles = hv.element.tiles.tile_sources[basemap_value]()
    return tiles.options(height=map_height, width=map_width).opts(gopts)

dynmap = hd.regrid(hv.DynamicMap(load_map))
combined = hv.DynamicMap(load_tiles) * dynmap

pn.Column(
  pn.WidgetBox(
    '## S2 L2A', 
    pn.Column(
        pn.Row(
            pn.Row('### Band', pn_band), 
            pn.Row('### Date',pn_date),
            pn.Row('#### Basemap',pn_basemaps)
            ), 
        pn.Row(
            pn.Row('',range_slider),
            pn.Row('#### Opacity', pn_opacity),
            pn.Row('#### Colormap', pn_colormap))      
  )), 
  combined
)


In [ ]:
hv.Layout([ts().relabel(name) for name, ts in hv.element.tiles.tile_sources.items()]).opts(
    opts.Tiles(xaxis=None, yaxis=None, width=225, height=225)).cols(4)

In [ ]:
from holoviews import opts

opts.defaults(
    opts.GridSpace(shared_xaxis=True, shared_yaxis=True),
    opts.Image(cmap='binary_r', width=800, height=700),
    opts.Labels(text_color='white', text_font_size='8pt', text_align='left', text_baseline='bottom'),
    opts.Path(color='white'),
    opts.Spread(width=900),
    opts.Overlay(show_legend=True))
# set the parameter for spectra extraction
hv.extension('bokeh')
pn.extension()

visual.image_viewer().Rrs_date(Rrs_masked)

In [ ]:
import datashader as ds
from datashader import transfer_functions as tf 
from colorcet import palette


shaded = []
for name, raster in Rrs_masked.isel(wl=2).groupby('time'):
    img = tf.shade(raster.rio.reproject(3857,nodata=np.nan).squeeze(),cmap=cc.kbc)
    img.name = str(name)
    shaded.append(img)

In [ ]:
imgs = tf.Images(*shaded)
imgs.num_cols = 3
imgs

In [ ]:
from holoviews import opts

opts.defaults(
    opts.GridSpace(shared_xaxis=True, shared_yaxis=True),
    opts.Image(cmap='binary_r', width=800, height=700),
    opts.Labels(text_color='white', text_font_size='8pt', text_align='left', text_baseline='bottom'),
    opts.Path(color='white'),
    opts.Spread(width=900),
    opts.Overlay(show_legend=True))
# set the parameter for spectra extraction
hv.extension('bokeh')
pn.extension()

raster = Rrs_masked#.isel(time=-1,drop=True)
ds = hv.Dataset(raster.persist())
im= ds.to(hv.Image, ['x', 'y'], dynamic=True).opts(cmap= 'RdBu_r',colorbar=True)#.hist(bin_range=(0,0.02) ) 
widget = pn.widgets.RangeSlider(start=0, end=0.1,step=0.001)

jscode = """
    color_mapper.low = cb_obj.value[0];
    color_mapper.high = cb_obj.value[1];
"""
link = widget.jslink(im, code={'value': jscode})
pn.Column(widget, im)

In [ ]:

param = 'Rrs'
third_dim = 'wl'
raster = Rrs_masked.isel(time=1,drop=True)
wl= raster.wl.data
Nwl = len(wl)
ds = hv.Dataset(raster.persist())
im= ds.to(hv.Image, ['x', 'y'], dynamic=True).opts(cmap= 'RdBu_r',colorbar=True,clim=(0,None)).hist(bin_range=(0,0.02) ) 

polys = hv.Polygons([])
box_stream = hv.streams.BoxEdit(source=polys)
dmap, dmap_std=[],[]

def roi_curves(data,ds=ds):    
    return hv.NdOverlay(curves)

mean=hv.DynamicMap(roi_curves,streams=[box_stream])
std =hv.DynamicMap(roi_spreads, streams=[box_stream])    
hlines = hv.HoloMa



    return hv.NdOverlay(curves)

mean=hv.DynamicMap(roi_curves,streams=[box_stream])
std =hv.DynamicMap(roi_spreads, streams=[box_stream])    
hlines = hv.HoloMa
    if not data or not any(len(d) for d in data.values()):
        return hv.NdOverlay({0: hv.Curve([],'Wavelength (nm)', 'Rrs')})

    curves,envelope = {},{}
    data = zip(data['x0'], data['x1'], data['y0'], data['y1'])
    for i, (x0, x1, y0, y1) in enumerate(data):
    return hv.NdOverlay(curves)

mean=hv.DynamicMap(roi_curves,streams=[box_stream])
std =hv.DynamicMap(roi_spreads, streams=[box_stream])    
hlines = hv.HoloMa
    return hv.NdOverlay(curves)

mean=hv.DynamicMap(roi_curves,streams=[box_stream])
    return hv.NdOverlay(curves)

mean=hv.DynamicMap(roi_curves,streams=[box_stream])
std =hv.DynamicMap(roi_spreads, streams=[box_stream])    
hlines = hv.HoloMa
std =hv.DynamicMap(roi_spreads, streams=[box_stream])    
    return hv.NdOverlay(curves)

mean=hv.DynamicMap(roi_curves,streams=[box_stream])
std =hv.DynamicMap(roi_spreads, streams=[box_stream])    
hlines = hv.HoloMa
hlines = hv.HoloMa
        selection = ds.select(x=(x0, x1), y=(y0, y1))
        mean = selection.aggregate(third_dim, np.mean).data
    return hv.NdOverlay(curves)

mean=hv.DynamicMap(roi_curves,streams=[box_stream])
std =hv.DynamicMap(roi_spreads, streams=[box_stream])    
hlines = hv.HoloMa
        std = selection.aggregate(third_dim, np.std).data
    return hv.NdOverlay(curves)

mean=hv.DynamicMap(roi_curves,streams=[box_stream])
std =hv.DynamicMap(roi_spreads, streams=[box_stream])    
hlines = hv.HoloMa
        wl = mean.wl

        curves[i]= hv.Curve((wl,mean[param]),'Wavelength (nm)', 'Rrs') 

    return hv.NdOverlay(curves)


# a bit dirty to have two similar function, but holoviews does not like mixing Curve and Spread for the same stream
def roi_spreads(data,ds=ds):    
    if not data or not any(len(d) for d in data.values()):
        return hv.NdOverlay({0: hv.Curve([],'Wavelength (nm)', 'Rrs')})

    curves,envelope = {},{}
    return hv.NdOverlay(curves)

mean=hv.DynamicMap(roi_curves,streams=[box_stream])
std =hv.DynamicMap(roi_spreads, streams=[box_stream])    
hlines = hv.HoloMa
    return hv.NdOverlay(curves)

mean=hv.DynamicMap(roi_curves,streams=[box_stream])
std =hv.DynamicMap(roi_spreads, streams=[box_stream])    
hlines = hv.HoloMa
    data = zip(data['x0'], data['x1'], data['y0'], data['y1'])
    for i, (x0, x1, y0, y1) in enumerate(data):
        selection = ds.select(x=(x0, x1), y=(y0, y1))
        mean = selection.aggregate(third_dim, np.mean).data
        std = selection.aggregate(third_dim, np.std).data
        wl = mean.wl

        curves[i]=  hv.Spread((wl,mean[param],std[param]),fill_alpha=0.3)

    return hv.NdOverlay(curves)

mean=hv.DynamicMap(roi_curves,streams=[box_stream])
std =hv.DynamicMap(roi_spreads, streams=[box_stream])    
hlines = hv.HoloMap({wl[i]: hv.VLine(wl[i]) for i in range(Nwl)},third_dim )


In [ ]:

# visualize and play
graphs = ((mean* std *hlines).relabel('Rrs'))
layout = (im * polys +graphs    ).opts(
    opts.Curve(width=600, framewise=True,xlim=(400,1000)), 
    opts.Polygons(fill_alpha=0.2, color='green',line_color='black'), 
    opts.VLine(color='black')).cols(2)
layout 

# Check surface rugosity via sunglint BRDF

In [ ]:

raster = BRDFg#.isel(time=-1,drop=True)
ds = hv.Dataset(raster.persist())
im= ds.to(hv.Image, ['x', 'y'], dynamic=True).opts(cmap= 'gray',colorbar=True)#.hist(bin_range=(0,0.02) ) 
widget = pn.widgets.RangeSlider(start=0, end=0.01,step=0.0005)

jscode = """
    color_mapper.low = cb_obj.value[0];
    color_mapper.high = cb_obj.value[1];
"""
link = widget.jslink(im, code={'value': jscode})
pn.Column(widget, im)

In [ ]:

shaded = []
for name, raster in BRDFg.groupby('time'):
    img = tf.shade(raster.squeeze(),cmap=cc.gray)
    img.name = str(name)
    shaded.append(img)

imgs = tf.Images(*shaded)
imgs.num_cols = 5
imgs

## Check blue over green ratio for Chl retrieval with OC2 from NASA
$log_{10}(chlor\_a) = a_0 + \sum\limits_{i=1}^4 a_i \left(log_{10}\left(\frac{R_{rs}(\lambda_{blue})}{R_{rs}(\lambda_{green})}\right)\right)^i$

In [ ]:
# NASA OC2 fro MODIS; bands 488, 547 nm
a = [0.2500,-2.4752,1.4061,-2.8233,0.5405]
# NASA OC2 for OCTS; bands 490, 565 nm
a = [0.2236,-1.8296,1.9094,-2.9481,-0.1718]

ratio = np.log10(Rrs_masked.isel(wl=1)/Rrs_masked.isel(wl=2))
logchl=0
for i in range(len(a)):
    logchl+=a[i]*ratio**i
chl = 10**(logchl)
chl.name='chl in mg.m-3 from OC2'

Set range of valid values

In [ ]:
chl = chl.where((chl >= 0) & (chl <= 80))
chl.persist()



In [ ]:
visual.image_viewer().param_date(chl,cmap='bgyw')

In [ ]:
raster = chl

shaded = []
for name, raster in chl.groupby('time'):
    img = tf.shade(raster.squeeze(),cmap=cc.bgyw, span=(0,10),how='log')
    img.name = str(name)
    shaded.append(img)

imgs = tf.Images(*shaded)
imgs.num_cols = 4
imgs

# CDOM retrieval based on Brezonik et al, 2015


In [ ]:
a = [1.872,-0.83]
acdom = np.exp(a[0] + a[1] * np.log(Rrs_masked.isel(wl=1)/Rrs_masked.isel(wl=5)))
acdom.name='CDOM absorption at 440 nm-1'
acdom= acdom.where((acdom >= 0) & (acdom <= 10))
acdom.persist()

In [ ]:
visual.image_viewer().param_date(acdom,cmap='bgyw')

# Total suspended particulate matter (SPM) from Nechad et al., 2010, 2016 formulation
spm in mg/L

In [ ]:
a = [610.94*np.pi, 0.2324/np.pi]
Rrs_ = Rrs_masked.isel(wl=3)
spm = a[0] * Rrs_ / (1 - ( Rrs_/ a[1]))
spm.name='CDOM absorption at 440 nm-1'
spm= spm.where((spm >= 0) & (spm <= 150))
spm.persist()

In [ ]:
visual.image_viewer().param_date(spm,cmap='bgyw')

In [ ]:

shaded = []
for name, raster in dc.Rrs.Rrs.isel(time=-1).groupby('wl'):
    img = tf.shade(raster,cmap=cc.kbc)
    img.name = '{:.2f}'.format(name)+' nm'
    shaded.append(img)

imgs = tf.Images(*shaded)
imgs.num_cols = 5
imgs

In [ ]:
shaded = []
for name, raster in BRDFg.groupby('time'):
    img = tf.shade(raster,cmap=cc.gray, span=(0,0.025),how='log')
    img.name = str(name)
    shaded.append(img)

imgs = tf.Images(*shaded)
imgs.num_cols = 4
imgs

# Play with time series

In [ ]:
raster = spm

param = raster.name
third_dim = 'time'
time= raster.time.data
Ntime = len(time)
ds = hv.Dataset(raster.persist())
im= ds.to(hv.Image, ['x', 'y'], dynamic=True).opts(cmap= 'RdBu_r',colorbar=True,clim=(0,100))#.hist(bin_range=(0,0.02) ) 

polys = hv.Polygons([])
box_stream = hv.streams.BoxEdit(source=polys)
dmap, dmap_std=[],[]

def roi_curves(data,ds=ds):    
    if not data or not any(len(d) for d in data.values()):
        return hv.NdOverlay({0: hv.Curve([],'time', param)})

    curves,envelope = {},{}
    data = zip(data['x0'], data['x1'], data['y0'], data['y1'])
    for i, (x0, x1, y0, y1) in enumerate(data):
        selection = ds.select(x=(x0, x1), y=(y0, y1))
        mean = selection.aggregate(third_dim, np.nanmean).data
        print(mean)
        #std = selection.aggregate(third_dim, np.std).data
        time = mean[third_dim]

        curves[i]= hv.Curve((time,mean[param]),'time', param) 

    return hv.NdOverlay(curves)


# a bit dirty to have two similar function, but holoviews does not like mixing Curve and Spread for the same stream
def roi_spreads(data,ds=ds):    
    if not data or not any(len(d) for d in data.values()):
        return hv.NdOverlay({0: hv.Curve([],'time', param)})

    curves,envelope = {},{}
    data = zip(data['x0'], data['x1'], data['y0'], data['y1'])
    for i, (x0, x1, y0, y1) in enumerate(data):
        selection = ds.select(x=(x0, x1), y=(y0, y1))
        mean = selection.aggregate(third_dim, np.nanmean).data
        std = selection.aggregate(third_dim, np.nanstd).data
        time = mean[third_dim]

        curves[i]=  hv.Spread((time,mean[param],std[param]),fill_alpha=0.3)

    return hv.NdOverlay(curves)

mean=hv.DynamicMap(roi_curves,streams=[box_stream])
std =hv.DynamicMap(roi_spreads, streams=[box_stream])    


In [ ]:
# visualize and play
graphs = ((mean*std ).relabel(param))
layout = (im * polys +graphs    ).opts(
    opts.Curve(width=600, framewise=True), 
    opts.Polygons(fill_alpha=0.2, color='green',line_color='black'), 
    ).cols(2)
layout 